In [1]:
from collections import Counter, defaultdict
import math

In [2]:
import nltk

In [3]:
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [4]:
def read_data(PATH):
    import os
    all_data = []
    for path, dirs, files in os.walk(PATH):
        temp = []
        for filename in files:
            filePath = path + '/' + filename
            f = open(filePath, 'r', encoding='utf-8')
            temp.append(parse_article([g for g in f]))

        if temp:
            all_data.append(temp)
    return all_data

In [5]:
def parse_article(f):
    """tokenize sentence and transform to lower and padding"""
    ret = [s.lower() for s in f]
    ret = [nltk.word_tokenize(s) for s in ret]
    for i, x in enumerate(ret):
        try:
            float(x)
            ret[i] = '<N>'
        except:
            pass
    ret = [['<s>'] + s + ['</s>'] for s in ret]
    return ret

In [6]:
def kn_model():
    """TODO: Bigram() + Pcontination()"""
    pass

In [7]:
def score_article():
    """TODO: get label that has highest scores"""
    pass

In [8]:
def gen_bigram(s):
    for i in range(len(s) - 1):
        yield (s[i], s[i + 1])

In [12]:
if __name__ == '__main__':
    print("load and preprocess training data")
    train_PATH = './r_train/'
    all_training_data = read_data(train_PATH)
    
    print('start training')
    # build kn model
    unigram = [Counter([word for art in articles for sent in art for word in sent]) for articles in all_training_data]
    bigram = [Counter(gram for art in articles for sent in art for gram in gen_bigram(sent)) for articles in all_training_data]
    l_bigram = [len(bg) for bg in bigram] # |{(w', w''): 0 < c(w', w'')}|
    categories = len(all_training_data)
    w0_bigram = [defaultdict(set) for i in range(categories)] # {w': 0 < c(w0, w')}
    w1_bigram = [defaultdict(set) for i in range(categories)] # {w': 0 < c(w', w1)}
    for cat, bg in enumerate(bigram):
        for w0, w1 in bg:
            w0_bigram[cat][w0].add(w1)
            w1_bigram[cat][w1].add(w0)


    print("start testing")
    test_PATH = './r_test'
    all_testing_data = read_data(test_PATH)
    test_data_len = sum(len(t) for t in all_testing_data)

    d = .75
    total_hits = [0] * categories
    for cat, articles in enumerate(all_testing_data):
        for art in articles:
            log_prob = [0] * categories
            for c in range(categories):
                for sent in art:
                    for word in gen_bigram(sent):
                        w0, w1 = word
                        c_w0 = unigram[c][w0]
                        if c_w0:
                            bigram_term = max((bigram[c][word] - d) / c_w0, 0)
                            lambda_term = d / c_w0 * len(w0_bigram[c][w0])
                            pconti_term = len(w1_bigram[c][w1]) / l_bigram[c]
                            prob = bigram_term + lambda_term * pconti_term + 0.15
                            log_prob[c] += prob
                        # else -
            pred_cat = log_prob.index(max(log_prob))
            total_hits[cat] += (cat == pred_cat)

    total_hit = sum(total_hits)
    print('Accuracy: ', total_hit / test_data_len)

load and preprocess training data
start training
start testing
Accuracy:  0.6046852122986823
